### PREPARATION

In [ ]:
import yaml
import logging
import pypsa
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
import os
import xarray as xr
import cartopy
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import sys
import matplotlib.lines as mlines

In [ ]:
logging.getLogger("pypsa.io").setLevel(logging.ERROR)
warnings.simplefilter(action='ignore', category=ResourceWarning)

# Load the UHS and woUHS networks
uhs = pypsa.Network("../../../pypsa-earth/results/UHS/postnetworks/elec_s_24_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")
woUHS = pypsa.Network("../../../pypsa-earth/results/woUHS/postnetworks/elec_s_24_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")

# Load geographic data for onshore regions and ports
regions_onshore = gpd.read_file("../../../pypsa-earth/resources/shapes/country_shapes.geojson")
ports = pd.read_csv("../../../pypsa-earth/resources/UHS/ports.csv")

# Create a GeoDataFrame for ports with point geometries
ports = gpd.GeoDataFrame(
    ports,
    geometry=gpd.points_from_xy(ports["x"], ports["y"]),
    crs="EPSG:4326"  # Coordinates are in WGS84
)

# Load GADM shapes and configuration file
gadm_shapes = gpd.read_file("../../../pypsa-earth/resources/shapes/gadm_shapes.geojson")
config = yaml.safe_load(open("../../../pypsa-earth/config.yaml"))

# Define paths for network and renewable profiles
network_path = "../../../pypsa-earth/networks/UHS/elec.nc"
solar_path = "../../../pypsa-earth/resources/UHS/renewable_profiles/profile_solar.nc"
onwind_path = "../../../pypsa-earth/resources/UHS/renewable_profiles/profile_onwind.nc"

# Get the bounding box for the onshore regions
country_coordinates = regions_onshore.total_bounds[[0, 2, 1, 3]]

# Define colors for scenarios
SCENARIO_COLORS = {"UHS": "#1f77b4", "woUHS": "#ff7f0e"}
tech_colors = config["plotting"]["tech_colors"]

# Normalize the carrier column/index to lowercase for easier matching
def get_color(carrier):
    # First, check for an exact match
    if carrier in tech_colors:
        return tech_colors[carrier]
    # Try matching with lowercase
    if carrier.lower() in tech_colors:
        return tech_colors[carrier.lower()]
    # Default color if no match is found
    return "lightgrey"

# Map colors to carriers in the UHS network
uhs.carriers["color"] = uhs.carriers.index.map(get_color)

warnings.simplefilter(action='default', category=ResourceWarning)

In [ ]:
# uhs1 = pypsa.Network("results/UHS1/postnetworks/elec_s_1_ec_lcopt_Co2L0.15-3h_3h_2050_0.094_NZ_199.8export.nc")
# woUHS1 = pypsa.Network("results/woUHS1/postnetworks/elec_s_1_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")
# uhs10 = pypsa.Network("results/UHS10/postnetworks/elec_s_10_ec_lcopt_Co2L0.15-3h_3h_2050_0.094_NZ_199.8export.nc")
# woUHS10 = pypsa.Network("results/woUHS10/postnetworks/elec_s_10_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")
# uhs100 = pypsa.Network("results/UHS100/postnetworks/elec_s_100_ec_lcopt_Co2L0.15-3h_3h_2050_0.094_NZ_199.8export.nc")
# woUHS100 = pypsa.Network("results/woUHS100/postnetworks/elec_s_100_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")
# uhswotm = pypsa.Network("results/UHSwotm/postnetworks/elec_s_24_ec_lcopt_Co2L0.15-3h_3h_2050_0.094_NZ_199.8export.nc")
# woUHSwotm = pypsa.Network("results/woUHSwotm/postnetworks/elec_s_24_ec_lcopt_Co2L0.15_3H_2050_0.094_NZ_199.8export.nc")

#### Show initial energy system design

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)

# --- Ensure all carriers have color and nice_name ---
missing_carriers = [
    'lignite', 'coal', 'solar rooftop', 'oil', 'gas',
    'residential rural solar thermal',
    'residential urban decentral solar thermal',
    'services rural solar thermal',
    'services urban decentral solar thermal',
    'urban central solar thermal',
    'ccgt'
]

carrier_color_mapping = {
    'lignite': 'coal',
    'solar rooftop': 'solar',
    'residential rural solar thermal': 'solar',
    'residential urban decentral solar thermal': 'solar',
    'services rural solar thermal': 'solar',
    'services urban decentral solar thermal': 'solar',
    'urban central solar thermal': 'solar',
    'ccgt': 'gas'
}

# Assign colors for missing carriers
for c, ref in carrier_color_mapping.items():
    uhs.carriers.loc[c, 'color'] = uhs.carriers.loc[ref, 'color']

# Set nice_names
uhs.carriers.loc['coal', 'nice_name'] = 'Coal'
uhs.carriers.loc['oil', 'nice_name'] = 'Oil'
uhs.carriers.loc['ccgt', 'nice_name'] = 'Gas'

for c, ref in carrier_color_mapping.items():
    if c not in ['ccgt']:  # ccgt already has its own name
        uhs.carriers.loc[c, 'nice_name'] = uhs.carriers.loc[ref, 'nice_name']

# --- Scale settings ---
bus_scale = 6e3 
line_scale = 6e3

# --- Legend settings ---
bus_sizes = [100, 1000]  # in MW
line_sizes = [100, 1000]  # in MW

# --- Remove "Load" carrier for plotting ---
if "load shedding" in uhs.carriers.index:
    uhs.carriers.drop("load shedding", inplace=True)

# --- Aggregate generator and storage capacities by bus and carrier ---
gen = uhs.generators[uhs.generators.carrier != "load shedding"].groupby(["bus", "carrier"]).p_nom.sum()
sto = uhs.storage_units.groupby(["bus", "carrier"]).p_nom.sum()
buses = pd.concat([gen, sto])

# --- Plotting ---
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={"projection": ccrs.PlateCarree()})

# --- Filter Links: only those that lie within GADM regions ---
# Extract the base part of the bus names
link_bus0_base = uhs.links.bus0.str.split("_AC").str[0]
link_bus1_base = uhs.links.bus1.str.split("_AC").str[0]

# Keep only links whose both ends are present in the GADM shapes
valid_links = uhs.links[
    link_bus0_base.isin(gadm_shapes.index) &
    link_bus1_base.isin(gadm_shapes.index) &
    (uhs.links.p_nom > 0)  # optional: only positive capacity
]

# overwrite temporarily for the plot
links_backup = uhs.links
uhs.links = valid_links

with plt.rc_context({"patch.linewidth": 0.}):
    uhs.plot(
        bus_sizes=buses / bus_scale,
        bus_alpha=0.7,
        line_widths=uhs.lines.s_nom / line_scale,
        link_widths=uhs.links.p_nom / line_scale,
        line_colors="teal",
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )

regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="white",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0,
)

# Set map extent
ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])

# --- Add legends ---
legend_kwargs = {"loc": "upper left", "frameon": False}
legend_circles_dict = {"bbox_to_anchor": (1, 0.67), "labelspacing": 2.5, **legend_kwargs}

add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1000} GW" for s in bus_sizes],
    legend_kw=legend_circles_dict,    
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1000} GW" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)

# --- Create patch legend for carriers (unique nice_name, non-zero capacity) ---
colors_legend = []
labels_legend = []
seen = set()
for c in uhs.carriers.index:
    name = uhs.carriers.loc[c, 'nice_name']
    color = uhs.carriers.loc[c, 'color']

    # Sum capacity over all buses for this carrier
    if c in buses.index.get_level_values('carrier'):
        val_total = buses.xs(c, level='carrier').sum()
    else:
        val_total = 0

    if name not in seen and val_total > 0:
        seen.add(name)
        colors_legend.append(color)
        labels_legend.append(name)

add_legend_patches(
    ax,
    colors_legend,
    labels_legend,
    legend_kw={"bbox_to_anchor": (1, 0), **legend_kwargs, "loc": "lower left"},
)

fig.tight_layout()
plt.show()

# Restore original links
uhs.links = links_backup
warnings.simplefilter(action='default', category=UserWarning)

#### Current Capacities of Energy Carriers and Usage of Storages

In [ ]:
# --- Extract data for current energy carriers ---
energy_carriers = (
    uhs.generators[~uhs.generators.carrier.str.contains("load", case=False)]
    .groupby("carrier")
    .p_nom.sum()
    .div(1e3)  # Convert to GW
    .sort_values(ascending=False)
)

# --- Extract data for current storages ---
storages = (
    uhs.stores.groupby("carrier")
    .e_nom.sum()
    .div(1e3)  # Convert to GWh
    .sort_values(ascending=False)
)

# --- Filter: keep only entries > 0 ---
energy_carriers = energy_carriers[energy_carriers > 0]
storages = storages[storages > 0]
storages = storages[~storages.index.isin(["solid biomass", "biogas"])]

# --- Plotting ---
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# --- Energy carriers ---
energy_colors = [get_color(c) for c in energy_carriers.index]
wedges1, _ = axes[0].pie(
    energy_carriers,
    labels=None,   # no labels directly in the chart
    startangle=90,
    colors=energy_colors
)
axes[0].set_title("Current Energy Carriers (GW)")

# Legend with capacity + share
labels1 = [
    f"{c} ({v:.1f} GW, {100*v/energy_carriers.sum():.1f}%)"
    for c, v in zip(energy_carriers.index, energy_carriers.values)
]
axes[0].legend(
    wedges1,
    labels1,
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1)
)

# --- Storages ---
storage_colors = [get_color(c) for c in storages.index]
wedges2, _ = axes[1].pie(
    storages,
    labels=None,
    startangle=90,
    colors=storage_colors
)
axes[1].set_title("Current Storages (GWh)")

# Legend with capacity + share
labels2 = [
    f"{c} ({v:.1f} GWh, {100*v/storages.sum():.1f}%)"
    for c, v in zip(storages.index, storages.values)
]
axes[1].legend(
    wedges2,
    labels2,
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1)
)

plt.tight_layout()
plt.show()


#### Where is it possible to build UHS in salt caverns?

In [ ]:
def extract_uhs_regions(n, gadm_shapes):
    """
    Extract UHS storage regions and capacities and merge them with GADM shapes.

    Parameters
    ----------
    n : PyPSA Network
        The network containing UHS storage units in uhs.stores.
    gadm_shapes : GeoDataFrame
        GADM regions with column 'GADM_ID' for merging.

    Returns
    -------
    GeoDataFrame
        GADM shapes enriched with:
        - 'Capacity': summed UHS capacity per region
        - 'UHS': boolean flag if region has UHS
    """

    # Filter UHS stores
    n_stores = n.stores[n.stores.carrier == "H2 UHS"].copy()

    # Extract region name from store index
    n_stores["region"] = n_stores.index.str.split("_AC").str[0]

    # Summed installed capacity (MW) by region
    n_capacity = (
        n_stores.groupby("region")["e_nom_opt"]
        .sum()
        .reset_index()
        .rename(columns={"e_nom_opt": "Capacity"})
    )

    # Merge with GADM shapes
    n_gadm = gadm_shapes.merge(
        n_capacity,
        how="left",
        left_on="GADM_ID",
        right_on="region"
    )

    # Mark regions that have UHS
    n_regions = n_stores["region"].unique()
    n_gadm["UHS"] = n_gadm["GADM_ID"].isin(n_regions)

    return n_gadm

uhs_gadm = extract_uhs_regions(uhs, gadm_shapes)
woUHS_gadm = extract_uhs_regions(woUHS, gadm_shapes)

In [ ]:
# Plot the map
fig, ax = plt.subplots(figsize=(12, 10))
uhs_gadm.plot(ax=ax, color="lightgrey", edgecolor="black", linewidth=0.5)

# Highlight regions with UHS
if not uhs_gadm[uhs_gadm["UHS"]].empty:
    uhs_gadm[uhs_gadm["UHS"]].plot(ax=ax, color="blue", label="UHS Regions")

# Add legend and title
blue_patch = mpatches.Patch(color='blue', label='UHS Regions')
grey_patch = mpatches.Patch(color='lightgrey', label='Other Regions')
ax.legend(handles=[grey_patch, blue_patch], loc="upper left")
ax.set_title("GADM Shapes with UHS Highlighted", fontsize=14)
plt.show()

#### Which hydrogen infrastructure does the model have in 2050?

In [ ]:
# --- Select, what should be shown ---
show_uhs = True
show_ports = True
show_pipelines = True
show_electrolyzers = True
show_woUHS = True

warnings.simplefilter(action='ignore')

def plot_infrastructure(ax, gadm, net, ports, show_uhs, show_pipelines,
                        show_electrolyzers, show_ports):
    """
    Draws the full hydrogen infrastructure on a given axis.

    Parameters
    ----------
    ax : matplotlib axis
        The axis to draw on.
    gadm : GeoDataFrame
        Regional shapes (uhs_gadm or woUHS_gadm).
    net : PyPSA network
        The network used for plotting (uhs or woUHS).
    ports : GeoDataFrame
        Port locations.
    show_uhs : bool
        Whether to show UHS capacities.
    show_pipelines : bool
        Whether to show pipelines.
    show_electrolyzers : bool
        Whether to show electrolyzers.
    show_ports : bool
        Whether to show ports.
    """

    # --- Base map ---
    gadm.plot(ax=ax, color="whitesmoke", edgecolor="black", linewidth=0.5)

    # --- UHS Capacities ---
    if show_uhs and "Capacity" in gadm.columns:
        gadm.dropna(subset=["Capacity"]).plot(
            ax=ax,
            column="Capacity",
            cmap="YlGnBu",
            edgecolor="black",
            linewidth=0.5,
            legend=False
        )

    # --- Pipelines ---
    if show_pipelines:
        h2_pipelines_rep = net.links[net.links.carrier == "H2 pipeline repurposed"]

        if not h2_pipelines_rep.empty:  # <-- check if there are pipelines
            # Build pipeline geometries
            def build_pipeline_geometry(row):
                x0, y0 = net.buses.loc[row.bus0, ["x", "y"]]
                x1, y1 = net.buses.loc[row.bus1, ["x", "y"]]
                return LineString([(x0, y0), (x1, y1)])

            pipelines_rep = h2_pipelines_rep.copy()
            pipelines_rep["geometry"] = pipelines_rep.apply(build_pipeline_geometry, axis=1)

            lines_gdf = gpd.GeoDataFrame(pipelines_rep, geometry="geometry", crs="EPSG:4326")

            if "p_nom_opt" in lines_gdf.columns:
                capacity = lines_gdf["p_nom_opt"]
                widths = np.interp(capacity, (capacity.min(), capacity.max()), (1.5, 7.0))
            else:
                widths = [1.5] * len(lines_gdf)

            # Draw pipelines
            for geom, w in zip(lines_gdf.geometry, widths):
                ax.plot(*geom.xy, color="#206bc7", linewidth=w, zorder=2)
        else:
            # No pipelines in this network, skip
            pass
        
    # --- Electrolyzers ---
    if show_electrolyzers:
        # Select all electrolyzer links
        electrolysis_links = net.links[net.links.carrier == "H2 Electrolysis"]
        electrolysis_ts = net.links_t.p1[electrolysis_links.index]
        bus_map = electrolysis_links.bus0

        # Group by bus and sum over time
        electrolysis_by_bus_ts = electrolysis_ts.T.groupby(bus_map).sum().T * 144
        total_elec = abs(electrolysis_by_bus_ts.sum() / 1e6)  # TWh

        # Coordinates for plotting
        coords = net.buses.loc[total_elec.index, ["x", "y"]]
        sizes = total_elec * 10

        # Draw electrolyzers
        ax.scatter(coords["x"], coords["y"], s=sizes,
                   c="lightblue", alpha=0.8, edgecolor="k")

    # --- Ports ---
    if show_ports:
        # Select ports inside the region shapes
        ports_within = gpd.sjoin(ports, gadm, how="inner", predicate="within")
        ports_within.plot(ax=ax, color="red", markersize=40, marker="o")

        # Draw port labels
        for x, y, name in zip(ports_within.geometry.x,
                              ports_within.geometry.y,
                              ports_within["name"]):
            ax.annotate(name, (x + 0.1, y + 0.1),
                        fontsize=9, color="red", ha="left", va="bottom")

    ax.set_axis_off()

# --- AUTOMATIC: one or two subplots ---
if show_woUHS:
    fig, axes = plt.subplots(1, 2, figsize=(26, 12))
    ax_left = axes[0]
    ax_right = axes[1]

    # --- LEFT PANEL: UHS network ---
    plot_infrastructure(
        ax_left, uhs_gadm, uhs, ports,
        show_uhs, show_pipelines, show_electrolyzers, show_ports
    )
    ax_left.set_title("Hydrogen Infrastructure (UHS Network)", fontsize=16)

    # --- RIGHT PANEL: network without UHS ---
    plot_infrastructure(
        ax_right, woUHS_gadm, woUHS, ports,
        False, show_pipelines, show_electrolyzers, show_ports
    )
    ax_right.set_title("Hydrogen Infrastructure (Without UHS)", fontsize=16)

else:
    # Single plot case
    fig, ax = plt.subplots(figsize=(14, 12))
    plot_infrastructure(
        ax, uhs_gadm, uhs, ports,
        show_uhs, show_pipelines, show_electrolyzers, show_ports
    )
    ax.set_title("Hydrogen Infrastructure", fontsize=16)

# --- Determine global min/max pipeline capacity from both networks ---
def get_pipeline_caps(net):
    df = net.links[net.links.carrier == "H2 pipeline repurposed"]
    if df.empty:
        return []
    return df["p_nom_opt"].values

all_caps = np.concatenate([
    get_pipeline_caps(uhs),
    get_pipeline_caps(woUHS)
]) if show_woUHS else get_pipeline_caps(uhs)

if len(all_caps) > 0:
    min_cap = all_caps.min()
    max_cap = all_caps.max()
else:
    min_cap, max_cap = 1, 10


# --- starting y-position for legends
legend_y = 0.9
spacing = 0.11

# Pipeline legend
if show_pipelines:
    legend_caps = [500, 1000, 5000, 10000]
    min_cap, max_cap = min(legend_caps), max(legend_caps)
    legend_widths = np.interp(legend_caps, (min_cap, max_cap), (1.5, 7.0))
    pipeline_handles = [
        mlines.Line2D([], [], color="#206bc7", linewidth=w, label=f"{cap/1000:.1f} GW")
        for cap, w in zip(legend_caps, legend_widths)
    ]
    leg1 = axes[1].legend(handles=pipeline_handles, title="Pipeline Capacity",
                           loc="upper left", bbox_to_anchor=(1.02, legend_y), fontsize=10)
    fig.add_artist(leg1)
    legend_y -= spacing

# Electrolyzer legend
if show_electrolyzers:
    example_caps = [10, 100]
    example_sizes = [c*10 for c in example_caps]
    bubble_handles = [
        plt.scatter([], [], s=s, facecolors="none", edgecolors="k", label=f"{cap} TWh")
        for s, cap in zip(example_sizes, example_caps)
    ]
    leg2 = axes[1].legend(handles=bubble_handles,
        title="Electrolysis Size",
        loc="upper left",
        bbox_to_anchor=(1.02, 0.75),
        borderaxespad=0,
        fontsize=11,
        scatterpoints=1,
        handletextpad=2,
        borderpad=1.5,
        labelspacing=2.0
        )
    fig.add_artist(leg2)
    legend_y -= spacing

   

# Ports legend
if show_ports:
    port_handle = [mlines.Line2D([], [], color="red", marker="o", linestyle="None",
                                 markersize=8, label="Port")]
    leg3 = axes[1].legend(handles=port_handle,
        loc="upper left",
        bbox_to_anchor=(1.02, 0.55),
        borderaxespad=0,
        fontsize=11)
    fig.add_artist(leg3)
    legend_y -= spacing

# UHS colorbar
if show_uhs and "Capacity" in uhs_gadm.columns and not uhs_gadm["Capacity"].dropna().empty:
    import matplotlib as mpl
    cmap = mpl.cm.YlGnBu
    norm = mpl.colors.Normalize(vmin=uhs_gadm["Capacity"].min(),
                               vmax=uhs_gadm["Capacity"].max())
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # Create a new axes for the colorbar on the right side
    cbar_ax = fig.add_axes([0.92, 0.15, 0.015, 0.7])  # [left, bottom, width, height]
    cbar = fig.colorbar(sm, cax=cbar_ax)
    cbar.set_label("UHS Capacity (MWh)")


plt.tight_layout()
plt.show()

#### Compare energy system with uhs and without

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)

# --- Common carrier settings for both networks ---
missing_carriers = [
    'lignite', 'coal', 'solar rooftop', 'oil', 'gas',
    'residential rural solar thermal',
    'residential urban decentral solar thermal',
    'services rural solar thermal',
    'services urban decentral solar thermal',
    'urban central solar thermal',
    'ccgt'
]

carrier_color_mapping = {
    'lignite': 'coal',
    'solar rooftop': 'solar',
    'residential rural solar thermal': 'solar',
    'residential urban decentral solar thermal': 'solar',
    'services rural solar thermal': 'solar',
    'services urban decentral solar thermal': 'solar',
    'urban central solar thermal': 'solar',
    'ccgt': 'gas'
}

def prepare_carriers(n):
    # Ensure columns exist
    if "color" not in n.carriers.columns:
        n.carriers["color"] = np.nan
    if "nice_name" not in n.carriers.columns:
        n.carriers["nice_name"] = n.carriers.index

    # Assign colors for missing carriers
    for c, ref in carrier_color_mapping.items():
        if c not in n.carriers.index:
            n.carriers.loc[c, ["color", "nice_name"]] = [np.nan, c.capitalize()]

        if ref in n.carriers.index and pd.notna(n.carriers.loc[ref, "color"]):
            n.carriers.loc[c, "color"] = n.carriers.loc[ref, "color"]
        else:
            n.carriers.loc[c, "color"] = "gray"  # fallback

    # Set custom nice_names
    if "coal" in n.carriers.index:
        n.carriers.loc["coal", "nice_name"] = "Coal"
    if "oil" in n.carriers.index:
        n.carriers.loc["oil", "nice_name"] = "Oil"
    if "ccgt" in n.carriers.index:
        n.carriers.loc["ccgt", "nice_name"] = "Gas"
        if pd.isna(n.carriers.loc["ccgt", "color"]) or n.carriers.loc["ccgt", "color"] == "":
            if "gas" in n.carriers.index and n.carriers.loc["gas", "color"] not in ["", np.nan]:
                n.carriers.loc["ccgt", "color"] = n.carriers.loc["gas", "color"]
            else:
                n.carriers.loc["ccgt", "color"] = "gray"

    # Set gray color for all remaining missing colors
    n.carriers["color"] = n.carriers["color"].replace("", np.nan).fillna("gray")

    # Remove "Load" if present
    if "Load shedding" in n.carriers.index:
        n.carriers.drop("Load shedding", inplace=True)

def plot_network(n, ax, title):
    # --- Aggregate capacities ---
    gen = n.generators[n.generators.carrier != "load shedding"].groupby(["bus", "carrier"]).p_nom_opt.sum()
    sto = n.storage_units.groupby(["bus", "carrier"]).p_nom_opt.sum()
    buses = pd.concat([gen, sto])

    # --- Filter links ---
    link_bus0_base = n.links.bus0.str.split("_AC").str[0]
    link_bus1_base = n.links.bus1.str.split("_AC").str[0]
    valid_links = n.links[
        link_bus0_base.isin(gadm_shapes.index) &
        link_bus1_base.isin(gadm_shapes.index) &
        (n.links.p_nom_opt > 0)
    ]
    links_backup = n.links
    n.links = valid_links

    with plt.rc_context({"patch.linewidth": 0.}):
        n.plot(
            bus_sizes=buses / bus_scale,
            bus_alpha=0.7,
            line_widths=n.lines.s_nom_opt / line_scale,
            link_widths=n.links.p_nom_opt / line_scale,
            line_colors="teal",
            ax=ax,
            margin=0.2,
            color_geomap=None,
        )

    regions_onshore.plot(
        ax=ax,
        facecolor="whitesmoke",
        edgecolor="white",
        aspect="equal",
        transform=ccrs.PlateCarree(),
        linewidth=0,
    )

    # Zoom the map
    ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])
    ax.set_title(title, fontsize=14)

    # Restore original links
    n.links = links_backup


# --- Scale settings ---
bus_scale = 6e5 
line_scale = 6e3
bus_sizes = [10000, 100000]  # in MW
line_sizes = [100, 1000]  # in MW

# --- Prepare both networks ---
prepare_carriers(uhs)
prepare_carriers(woUHS)

# --- Figure with two subplots ---
fig, axes = plt.subplots(
    1, 2, figsize=(18, 8), subplot_kw={"projection": ccrs.PlateCarree()}
)

plot_network(uhs, axes[0], "Scenario with UHS (2050)")
plot_network(woUHS, axes[1], "Scenario without UHS (2050)")

# --- Add legends once (on the right) ---
legend_kwargs = {"loc": "upper left", "frameon": False}
legend_circles_dict = {"bbox_to_anchor": (1.05, 0.67), "labelspacing": 2.5, **legend_kwargs}

add_legend_circles(
    axes[1],
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1000} GW" for s in bus_sizes],
    legend_kw=legend_circles_dict,    
)
add_legend_lines(
    axes[1],
    [s / line_scale for s in line_sizes],
    [f"{s / 1000} GW" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1.05, 0.8), **legend_kwargs},
)

# Carrier legend:
used_carriers = set(uhs.generators.carrier).union(set(uhs.storage_units.carrier))
colors_legend = []
labels_legend = []
seen = set()
for c in uhs.carriers.index:
    if c not in used_carriers:
        continue
    # Fallback for missing nice_name
    name = uhs.carriers.loc[c, "nice_name"]
    if pd.isna(name) or name == "":
        name = c.capitalize()
    # Fallback for missing color
    color = uhs.carriers.loc[c, "color"]
    if pd.isna(color) or color == "":
        color = "gray"
    if name not in seen:
        seen.add(name)
        colors_legend.append(color)
        labels_legend.append(name)

add_legend_patches(
    axes[1],
    colors_legend,
    labels_legend,
    legend_kw={"bbox_to_anchor": (1.05, 0), **legend_kwargs, "loc": "lower left"},
)

fig.tight_layout()
plt.show()

warnings.simplefilter(action="default", category=UserWarning)

#### Capacity extension of energy carriers

In [ ]:
# Calculate capacity expansions for uhs
optimal_capacity_uhs = uhs.statistics.optimal_capacity(comps=["Generator"]).droplevel(0).div(1e3)
installed_capacity_uhs = uhs.statistics.installed_capacity(comps=["Generator"]).droplevel(0).div(1e3)
generation_capacity_expansion_uhs = optimal_capacity_uhs - installed_capacity_uhs

# Calculate capacity expansions for woUHS
optimal_capacity_wouhs = woUHS.statistics.optimal_capacity(comps=["Generator"]).droplevel(0).div(1e3)
installed_capacity_wouhs = woUHS.statistics.installed_capacity(comps=["Generator"]).droplevel(0).div(1e3)
generation_capacity_expansion_wouhs = optimal_capacity_wouhs - installed_capacity_wouhs

# --- Prepare data ---
df_expansion = pd.DataFrame({
    "Scenario with UHS": generation_capacity_expansion_uhs,
    "Scenario without UHS": generation_capacity_expansion_wouhs
}).fillna(0)  # Fill missing carriers with 0 if they appear only in one scenario

# --- Plot ---
fig, ax = plt.subplots(figsize=(12, 6))

width = 0.35  # Width of the bars
x = np.arange(len(df_expansion.index))  # Carrier positions on x-axis

# Bars for UHS scenario
ax.bar(x - width/2, df_expansion["Scenario with UHS"], width, color=SCENARIO_COLORS["UHS"], label="Scenario with UHS")

# Bars for woUHS scenario
ax.bar(x + width/2, df_expansion["Scenario without UHS"], width, color=SCENARIO_COLORS["woUHS"], label="Scenario without UHS")

# Axis labels and title
ax.set_xticks(x)
ax.set_xticklabels(df_expansion.index, rotation=45, ha="right")
ax.set_ylabel("Capacity Expansion (GW)")
ax.set_title("Generator Capacity Expansion Comparison")

# Legend
ax.legend()

plt.tight_layout()
plt.show()

#### Capacity extension of storages

In [ ]:
# Extract storage capacity extensions for uhs and woUHS
uhs_storage_extension = uhs.stores.groupby("carrier").e_nom_opt.sum().div(1e3)  # Convert to GWh
wouhs_storage_extension = woUHS.stores.groupby("carrier").e_nom_opt.sum().div(1e3)  # Convert to GWh

# Filter out carriers with zero capacity extension
uhs_storage_extension = uhs_storage_extension[uhs_storage_extension > 0]
wouhs_storage_extension = wouhs_storage_extension[wouhs_storage_extension > 0]

# Combine for consistent order
all_carriers = sorted(set(uhs_storage_extension.index) | set(wouhs_storage_extension.index))
uhs_storage_extension = uhs_storage_extension.reindex(all_carriers, fill_value=0)
wouhs_storage_extension = wouhs_storage_extension.reindex(all_carriers, fill_value=0)

# --- Plot ---
fig, ax = plt.subplots(figsize=(12, 6))

width = 0.35  # Width of the bars
x = np.arange(len(all_carriers))  # Carrier positions on x-axis

# Bars for UHS scenario
ax.bar(x - width/2, uhs_storage_extension, width, color=SCENARIO_COLORS["UHS"], label="Scenario with UHS")

# Bars for woUHS scenario
ax.bar(x + width/2, wouhs_storage_extension, width, color=SCENARIO_COLORS["woUHS"], label="Scenario without UHS")

# Axis labels and title
ax.set_xticks(x)
ax.set_xticklabels(all_carriers, rotation=45, ha="right")
ax.set_ylabel("Capacity Extension (GWh)")
ax.set_title("Storage Capacity Extension Comparison")

# Legend
ax.legend()

plt.tight_layout()
plt.show()

#### Hydrogen related extension

In [ ]:
# --- Extract hydrogen-related capacities ---#
# Extract hydrogen-related capacity extensions for UHS and woUHS
uhs_hydrogen_extension = uhs.statistics.energy_balance().loc[:, :, "Hydrogen Storage"].groupby("carrier").sum().div(1e6)  # Convert to TWh
wouhs_hydrogen_extension = woUHS.statistics.energy_balance().loc[:, :, "Hydrogen Storage"].groupby("carrier").sum().div(1e6)  # Convert to TWh
# Combine for consistent order
all_hydrogen_carriers = sorted(set(uhs_hydrogen_extension.index) | set(wouhs_hydrogen_extension.index))
uhs_hydrogen_extension = uhs_hydrogen_extension.reindex(all_hydrogen_carriers, fill_value=0)
wouhs_hydrogen_extension = wouhs_hydrogen_extension.reindex(all_hydrogen_carriers, fill_value=0)

# --- Plot ---
fig, ax = plt.subplots(figsize=(12, 6))

width = 0.35  # Width of the bars
x = np.arange(len(all_hydrogen_carriers))  # Carrier positions on x-axis

# Bars for UHS scenario
ax.bar(x - width/2, uhs_hydrogen_extension, width, color=SCENARIO_COLORS["UHS"], label="Scenario with UHS")

# Bars for woUHS scenario
ax.bar(x + width/2, wouhs_hydrogen_extension, width, color=SCENARIO_COLORS["woUHS"], label="Scenario without UHS")

# Axis labels and title
ax.set_xticks(x)
ax.set_xticklabels(all_hydrogen_carriers, rotation=45, ha="right")
ax.set_ylabel("Production and Supply (TWh)")
ax.set_title("Hydrogen Production and Supply Comparison")

# Legend
ax.legend()

plt.tight_layout()
plt.show()

##### Energy Balance

In [ ]:
def compute_energy_balance(n: pypsa.Network) -> pd.DataFrame:
    # Rename columns for better readability
    rename_cols = {
        "-": "Load",
        "load": "load shedding",
    }

    # Calculate the energy balance, aggregate by carrier, and convert MWh to TWh
    energy_balance = (
        n.statistics.energy_balance()
        .loc[:, :, :]
        .groupby("carrier")
        .sum()
        .div(1e6)  # Convert MWh to TWh
        .to_frame()
        .T
        .rename(columns=rename_cols)
    )

    # Remove carriers with no energy flow
    energy_balance = energy_balance.loc[:, (energy_balance != 0).any(axis=0)]

    # Uncomment the following lines to filter out values below 0.1 TWh
    # energy_balance = energy_balance.loc[:, (energy_balance.abs() >= 0.1).any(axis=0)]
    return energy_balance


# --- Compute data ---
df_uhs = compute_energy_balance(uhs)
df_wouhs = compute_energy_balance(woUHS)

# Harmonize columns between the two dataframes
all_cols = sorted(set(df_uhs.columns) | set(df_wouhs.columns))
df_uhs = df_uhs.reindex(columns=all_cols, fill_value=0)
df_wouhs = df_wouhs.reindex(columns=all_cols, fill_value=0)

# Sort columns by total sum
order = (df_uhs.add(df_wouhs, fill_value=0)).iloc[0].sort_values(ascending=False).index.tolist()
df_uhs = df_uhs[order]
df_wouhs = df_wouhs[order]

# --- Assign colors ---
colors = [get_color(c) for c in order]

# --- Plot ---
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Plot energy balance for UHS scenario
df_uhs.plot.bar(
    stacked=True, 
    ax=axes[0], 
    legend=False, 
    title="Scenario with UHS – Energy Balance (TWh)", 
    color=colors
)
axes[0].set_xlabel("")
axes[0].set_ylabel("TWh")

# Plot energy balance for woUHS scenario
df_wouhs.plot.bar(
    stacked=True, 
    ax=axes[1], 
    legend=False,  # Legend will be added manually
    title="Scenario without UHS – Energy Balance (TWh)", 
    color=colors
)
axes[1].set_xlabel("")
axes[1].set_ylabel("")

# --- Manually build the legend ---
labels = []
handles = []
for c, col in zip(order, colors):
    val_uhs = df_uhs.iloc[0][c]
    val_wouhs = df_wouhs.iloc[0][c]
    # Only include carriers with significant values in the legend
    if np.abs(val_uhs) > 10 or np.abs(val_wouhs) > 10:
        labels.append(f"{c} ({val_uhs:.1f}, {val_wouhs:.1f} TWh)")
        handles.append(plt.Rectangle((0, 0), 1, 1, color=col))

axes[1].legend(
    handles, 
    labels, 
    bbox_to_anchor=(1.02, 1), 
    loc="upper left", 
    borderaxespad=0
)
plt.tight_layout()
plt.show()

##### Heating Supply Distribution

In [ ]:
logging.getLogger("pypsa.statistics").setLevel(logging.ERROR)
warnings.simplefilter(action='ignore', category=UserWarning)
# Define which heating technologies to report (match by substring, case-insensitive) 
heating_types = ["Gas Boiler", "Heat Pump", "Resistive Heater", "CHP"]

def heating_distribution(n: pypsa.Network, tech_labels=heating_types):
    """
    Returns a Series indexed by tech_labels with MWh totals for each label.
    Matches each label against the 'carrier' level (case-insensitive).
    """
    # Aggregate supply for Links over the full time horizon
    s = (
        n.statistics.supply(comps=["Link"], aggregate_time="sum")
        .loc[lambda x: x.index.get_level_values("carrier").str.contains("heat|boiler|pump|chp", case=False)]
    )

    # Helper: sum all rows whose 'carrier' contains a keyword
    def sum_for(keyword: str) -> float:
        mask = s.index.get_level_values("carrier").str.contains(keyword, case=False, na=False)
        if not mask.any():
            return 0.0
        # Sum across all matching rows and all columns -> scalar (MWh)
        return float(s.loc[mask].sum(numeric_only=True).sum())

    data = {label: sum_for(label) for label in tech_labels}
    return pd.Series(data, index=tech_labels)

# Compute totals (MWh)
uhs_mwh = heating_distribution(uhs)
wouhs_mwh = heating_distribution(woUHS)

#  Combine for consistent order and to keep zeros explicit 
df = pd.DataFrame({"Scenario with UHS": uhs_mwh, "Scenario without UHS": wouhs_mwh})

# Optional: convert to TWh for readability (uncomment next line if desired)
df = df / 1e6

# --- Plot side-by-side bar charts ---
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# Left: UHS scenario
axes[0].bar(df.index, df["Scenario with UHS"], color=SCENARIO_COLORS["UHS"])
axes[0].set_title("UHS – Heating Supply Distribution")
axes[0].set_ylabel("TWh")
axes[0].set_xticklabels(df.index, rotation=20, ha="right")

# Right: woUHS scenario
axes[1].bar(df.index, df["Scenario without UHS"], color=SCENARIO_COLORS["woUHS"])
axes[1].set_title("woUHS – Heating Supply Distribution")
axes[1].set_xticklabels(df.index, rotation=20, ha="right")

# --- Align y-axis for fair comparison ---
ymax = max(df.values.max(), 1)
axes[0].set_ylim(0, ymax * 1.10)
axes[1].set_ylim(0, ymax * 1.10)

# --- Add value labels on top of bars ---
def add_labels(ax, values):
    for x, v in enumerate(values):
        ax.text(x, v, f"{v:,.0f}", va="bottom", ha="center", fontsize=9)

add_labels(axes[0], df["Scenario with UHS"].values)
add_labels(axes[1], df["Scenario without UHS"].values)

plt.tight_layout()
plt.show()
warnings.simplefilter(action='default', category=UserWarning)

##### Transport Supply Distribution

In [ ]:
logging.getLogger("pypsa.statistics").setLevel(logging.ERROR)
warnings.simplefilter(action='ignore', category=UserWarning)
# Only the mentioned technologies below are considered here
transport_types = [
    "BEV charger",
    "V2G",
    "H2 Fuel Cell",
    "H2 turbine",
    "H2 Electrolysis",
    "H2 UHS charger",
    "H2 UHS discharger",
    "Fischer-Tropsch",
    "solid biomass transport",
]

def transport_distribution(n: pypsa.Network, tech_labels=transport_types) -> pd.Series:
    """
    Aggregate transport-related supply from Link components (MWh totals per label).
    Each label matches rows whose index 'carrier' contains that label (case-insensitive).
    """
    s = (
        n.statistics.supply(comps=["Link"], aggregate_time="sum")
        # keep only rows that look like transport-ish carriers to speed things up
        .loc[lambda x: x.index.get_level_values("carrier").str.contains(
            "bev|v2g|h2|fischer|biomass|transport|turbine|fuel cell|electrolysis|uhs",
            case=False, na=False
        )]
    )

    def sum_for(keyword: str) -> float:
        mask = s.index.get_level_values("carrier").str.contains(keyword, case=False, na=False)
        if not mask.any():
            return 0.0
        # sum across matching rows and all numeric columns -> scalar MWh
        return float(s.loc[mask].sum(numeric_only=True).sum())

    return pd.Series({label: sum_for(label) for label in tech_labels}, index=tech_labels)

#  Compute totals (MWh) and harmonize order 
uhs_mwh = transport_distribution(uhs)
wouhs_mwh = transport_distribution(woUHS)
df = pd.DataFrame({"UHS": uhs_mwh, "woUHS": wouhs_mwh})

# Optional: convert to TWh for readability
# df = df / 1e6

# Order bars by combined magnitude (largest first) for easy visual comparison
order = (df.sum(axis=1)).sort_values(ascending=False).index.tolist()
df = df.loc[order]

#  Plot side-by-side bar charts (no custom colors) 
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

axes[0].bar(df.index, df["UHS"], color=SCENARIO_COLORS["UHS"])
axes[0].set_title("UHS – Transport Supply Distribution")
axes[0].set_ylabel("MWh")  # change to "TWh" if you convert above
axes[0].set_xticklabels(df.index, rotation=25, ha="right")

axes[1].bar(df.index, df["woUHS"], color=SCENARIO_COLORS["woUHS"])
axes[1].set_title("woUHS – Transport Supply Distribution")
axes[1].set_ylabel("")
axes[1].set_xticklabels(df.index, rotation=25, ha="right")

# same y-limit for fair comparison
ymax = max(df.max())
axes[0].set_ylim(0, ymax * 1.10 if ymax > 0 else 1)

# simple value labels
def add_labels(ax, vals):
    for i, v in enumerate(vals):
        ax.text(i, v, f"{v:,.0f}", va="bottom", ha="center", fontsize=9)

add_labels(axes[0], df["UHS"].values)
add_labels(axes[1], df["woUHS"].values)

plt.tight_layout()
plt.show()
warnings.simplefilter(action='default', category=UserWarning)

#### Hydrogen prices

##### CAPEX

In [ ]:
def hydrogen_capex_mio(n: pypsa.Network) -> float:
    """
    Sum CAPEX for hydrogen-related Links (index contains 'H2', case-insensitive).
    Works whether index is MultiIndex with 'carrier' or a simple Index.
    Returns Mio. €.
    """
    capex = n.statistics.capex(comps=["Link"])  # pandas Series of EUR
    idx = capex.index

    # Get carrier labels robustly
    if isinstance(idx, pd.MultiIndex):
        if "carrier" in (idx.names or []):
            carriers = idx.get_level_values("carrier")
        else:
            # Fall back to the last level if names aren't present
            carriers = idx.get_level_values(-1)
    else:
        carriers = pd.Index(idx)

    mask = carriers.astype(str).str.contains("H2", case=False, na=False)
    return float(capex[mask].sum()) / 1e6  # -> Mio. €

#  Compute hydrogen CAPEX (Mio. €) 
uhs_h2_capex = round(hydrogen_capex_mio(uhs), 2)
wouhs_h2_capex = round(hydrogen_capex_mio(woUHS), 2)

#  Bar plot side-by-side 
labels = ["UHS", "woUHS"]
values = [uhs_h2_capex, wouhs_h2_capex]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4.5))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen CAPEX – UHS vs woUHS")
ax.set_ylabel("Mio. €")
ax.set_ylim(0, max(values)*1.15 if max(values) > 0 else 1)

# Value labels
for rect, v in zip(bars, values):
    ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
            f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### OPEX

In [ ]:
def hydrogen_opex_mio(n: pypsa.Network) -> float:
    """
    Sum OPEX for hydrogen-related Links (carrier contains 'H2', case-insensitive).
    Works with both MultiIndex and simple Index.
    Returns Mio. €.
    """
    opex = n.statistics.opex(comps=["Link"])  # Series of EUR
    idx = opex.index

    if isinstance(idx, pd.MultiIndex):
        if "carrier" in (idx.names or []):
            carriers = idx.get_level_values("carrier")
        else:
            carriers = idx.get_level_values(-1)
    else:
        carriers = pd.Index(idx)

    mask = carriers.astype(str).str.contains("H2", case=False, na=False)
    return float(opex[mask].sum()) / 1e6  # -> Mio. €

#  Compute hydrogen OPEX (Mio. €) 
uhs_h2_opex = round(hydrogen_opex_mio(uhs), 2)
wouhs_h2_opex = round(hydrogen_opex_mio(woUHS), 2)

#  Plot side-by-side bar chart 
labels = ["UHS", "woUHS"]
values = [uhs_h2_opex, wouhs_h2_opex]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4.5))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen OPEX – UHS vs woUHS")
ax.set_ylabel("Mio. €")
ax.set_ylim(0, max(values)*1.15 if max(values) > 0 else 1)

# Add value labels
for rect, v in zip(bars, values):
    ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
            f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### LCOH

In [ ]:
logging.getLogger("pypsa.statistics").setLevel(logging.ERROR)
def _sum_matching(series_or_df, pattern: str) -> float:
    """Sum entries whose (multi)index 'carrier' (or last level) contains pattern (case-insensitive)."""
    idx = series_or_df.index
    if isinstance(idx, pd.MultiIndex):
        name_list = list(idx.names) if idx.names is not None else []
        level = "carrier" if "carrier" in name_list else idx.nlevels - 1
        carriers = idx.get_level_values(level).astype(str)
    else:
        carriers = pd.Index(idx).astype(str)
    mask = carriers.str.contains(pattern, case=False, na=False)
    return float(series_or_df.loc[mask].sum())

def _h2_capex_mio(n: pypsa.Network) -> float:
    # CAPEX (EUR) for Link entries whose carrier mentions 'H2'
    capex = n.statistics.capex(comps=["Link"])  # Series in EUR
    return _sum_matching(capex, "H2") / 1e6

def _h2_opex_mio(n: pypsa.Network) -> float:
    # OPEX (EUR) for Link entries whose carrier mentions 'H2'
    opex = n.statistics.opex(comps=["Link"])  # Series in EUR
    return _sum_matching(opex, "H2") / 1e6

def _h2_supply_mwh(n: pypsa.Network) -> float:
    # Supply (MWh) from Links whose carrier is 'H2 Electrolysis'
    # (electrolyser output is H2; adapt the pattern if your model names differ)
    supply = n.statistics.supply(comps=["Link"], aggregate_time="sum")  # DataFrame (columns by metric)
    # Some PyPSA versions return a Series—coerce to Series over all metrics if needed:
    if isinstance(supply, pd.Series):
        return _sum_matching(supply, r"^H2 Electrolysis$")
    # When DataFrame with multiple columns, sum across columns to get a scalar
    supply_series = supply.sum(axis=1, numeric_only=True)
    return _sum_matching(supply_series, r"^H2 Electrolysis$")

def _h2_electricity_cost_eur(n: pypsa.Network) -> float:
    """
    Compute electricity cost (€) to run H2 Electrolysis by multiplying each electrolyser's
    bus0 price with its electrical input (-p0), snapshot-weighted, and summing.
    """
    # Identify electrolyser links and their buses
    elec_links = n.links.index[n.links.carrier.str.fullmatch(r"H2 Electrolysis", case=False, na=False)]
    if len(elec_links) == 0:
        return 0.0

    # Prices on bus0 for each electrolyser
    bus0 = n.links.loc[elec_links, "bus0"]
    # Get marginal prices for those buses (align by snapshot)
    price_df = n.buses_t.marginal_price[bus0.values]
    price_df.columns = elec_links  # map each column to link name

    # Electrical input is -p0 (MW). Use p0 (usually <= 0 for consumption).
    p0_df = n.links_t.p0[elec_links]  # MW
    elec_consumption = -p0_df  # MW, non-negative

    # Snapshot weighting (usually hours)
    w = n.snapshot_weightings.generators  # Series indexed by snapshots

    # Elementwise multiply price (€/MWh) * consumption (MW) = €/h, then weight by hours and sum
    cost_per_snap = (price_df * elec_consumption).sum(axis=1)  # €/h assuming €/MWh * MW
    total_cost_eur = float((cost_per_snap * w).sum())
    return total_cost_eur

def compute_lcoh_eur_per_mwh(n: pypsa.Network) -> float:
    capex_mio = _h2_capex_mio(n)
    opex_mio  = _h2_opex_mio(n)
    e_cost_eur = _h2_electricity_cost_eur(n)
    supply_mwh = _h2_supply_mwh(n)

    # Convert Mio € to € for CAPEX & OPEX; keep e_cost in €
    numerator_eur = capex_mio * 1e6 + opex_mio * 1e6 + e_cost_eur
    if supply_mwh <= 0:
        return float("nan")
    return round(numerator_eur / supply_mwh, 2)

# - Compute LCOH -
lcoh_uhs = compute_lcoh_eur_per_mwh(uhs)
lcoh_wouhs = compute_lcoh_eur_per_mwh(woUHS)

# - Plot side-by-side -
labels = ["UHS", "woUHS"]
values = [lcoh_uhs, lcoh_wouhs]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4.5))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Levelized Cost of Hydrogen (€/MWh) – UHS vs woUHS")
ax.set_ylabel("€/MWh")
ax.set_ylim(0, max(v for v in values if pd.notna(v)) * 1.15 if all(pd.notna(v) for v in values) else 1)

# value labels
for rect, v in zip(bars, values):
    if pd.notna(v):
        ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
                f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Supply

In [ ]:
logging.getLogger("pypsa.statistics").setLevel(logging.ERROR)
def hydrogen_supply_twh(n: pypsa.Network) -> float:
    """
    Sum supply for hydrogen-related Links (carrier contains 'H2').
    Returns TWh.
    """
    supply = n.statistics.supply(comps=["Link"], aggregate_time="sum")  # MWh
    idx = supply.index
    if isinstance(idx, pd.MultiIndex):
        # if multi-index, use 'carrier' if available, otherwise last level
        level = "carrier" if "carrier" in (idx.names or []) else idx.nlevels - 1
        carriers = idx.get_level_values(level).astype(str)
    else:
        carriers = pd.Index(idx).astype(str)

    mask = carriers.str.contains("H2", case=False, na=False)
    return float(supply[mask].sum().sum()) / 1e6  # -> TWh

#  Compute hydrogen supply (TWh) 
uhs_h2_supply = round(hydrogen_supply_twh(uhs), 2)
wouhs_h2_supply = round(hydrogen_supply_twh(woUHS), 2)

#  Plot side-by-side bar chart 
labels = ["UHS", "woUHS"]
values = [uhs_h2_supply, wouhs_h2_supply]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4.5))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen Supply – UHS vs woUHS")
ax.set_ylabel("TWh")
ax.set_ylim(0, max(values)*1.15 if max(values) > 0 else 1)

# Value labels
for rect, v in zip(bars, values):
    ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
            f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Revenue

In [ ]:
def hydrogen_revenue_mio(n: pypsa.Network) -> float:
    """
    Sum Revenue (€) for hydrogen-related Links (carrier contains 'H2').
    Works whether the index is MultiIndex with 'carrier' or a simple Index.
    Returns Mio. €.
    """
    revenue = n.statistics.revenue(comps=["Link"])  # Series in EUR
    idx = revenue.index

    if isinstance(idx, pd.MultiIndex):
        level = "carrier" if "carrier" in (idx.names or []) else idx.nlevels - 1
        carriers = idx.get_level_values(level).astype(str)
    else:
        carriers = pd.Index(idx).astype(str)

    mask = carriers.str.contains("H2", case=False, na=False)
    return float(revenue[mask].sum()) / 1e6  # -> Mio. €

#  Compute hydrogen revenue (Mio. €) 
uhs_h2_rev = round(hydrogen_revenue_mio(uhs), 2)
wouhs_h2_rev = round(hydrogen_revenue_mio(woUHS), 2)

#  Plot side-by-side bar chart 
labels = ["UHS", "woUHS"]
values = [uhs_h2_rev, wouhs_h2_rev]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4.5))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen Revenue (Links) – UHS vs woUHS")
ax.set_ylabel("Mio. €")
ax.set_ylim(0, max(values)*1.15 if max(values) > 0 else 1)

# Value labels
for rect, v in zip(bars, values):
    ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
            f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Hydrogen Market

In [ ]:
def hydrogen_market_value(n: pypsa.Network) -> float:
    """
    Extract yearly hydrogen market value for H2 Electrolysis (Link).
    Returns value in €.
    """
    stats = n.statistics()
    try:
        return float(stats.loc["Link", "H2 Electrolysis"]["Market Value"])
    except Exception:
        # fallback: filter revenue-like table if Market Value not in statistics() directly
        mv = n.statistics()
        idx = mv.index
        if isinstance(idx, pd.MultiIndex):
            if "carrier" in (idx.names or []):
                carriers = idx.get_level_values("carrier").astype(str)
            else:
                carriers = idx.get_level_values(-1).astype(str)
        else:
            carriers = pd.Index(idx).astype(str)
        mask = carriers.str.fullmatch(r"H2 Electrolysis", case=False, na=False)
        if "Market Value" in mv.columns:
            return float(mv.loc[mask, "Market Value"].sum())
        else:
            return float("nan")

#  Compute hydrogen market values 
uhs_mv = round(hydrogen_market_value(uhs), 2)
woUHS_mv = round(hydrogen_market_value(woUHS), 2)

#  Bar plot comparison 
labels = ["UHS", "woUHS"]
values = [uhs_mv, woUHS_mv]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen Market Value – UHS vs woUHS")
ax.set_ylabel("€")

for rect, v in zip(bars, values):
    if pd.notna(v):
        ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
                f"{v:,.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Hydrogen costs

In [ ]:
def hydrogen_cost_per_unit(n: pypsa.Network) -> float:
    """
    Compute hydrogen production cost per unit:
    (CAPEX + OPEX) / Supply
    Returns Mio. € / TWh.
    """
    stats = n.statistics()

    try:
        capex_mio = stats.loc["Link", "H2 Electrolysis"].loc["Capital Expenditure"] / 1e6
        opex_mio  = stats.loc["Link", "H2 Electrolysis"].loc["Operational Expenditure"] / 1e6
        supply_twh = stats.loc["Link", "H2 Electrolysis"].loc["Supply"].sum() / 1e6
    except Exception:
        capex = n.statistics.capex(comps=["Link"])
        opex  = n.statistics.opex(comps=["Link"])
        supply = n.statistics.supply(comps=["Link"], aggregate_time="sum")

        def carriers_of(idx):
            if isinstance(idx, pd.MultiIndex):
                lvl = "carrier" if "carrier" in (idx.names or []) else idx.nlevels - 1
                return idx.get_level_values(lvl).astype(str)
            return pd.Index(idx).astype(str)

        capex_mask = carriers_of(capex.index).str.fullmatch(r"H2 Electrolysis", case=False, na=False)
        opex_mask  = carriers_of(opex.index).str.fullmatch(r"H2 Electrolysis", case=False, na=False)
        sup_mask   = carriers_of(supply.index).str.fullmatch(r"H2 Electrolysis", case=False, na=False)

        capex_mio = float(capex[capex_mask].sum()) / 1e6
        opex_mio  = float(opex[opex_mask].sum()) / 1e6
        if isinstance(supply, pd.DataFrame):
            supply_twh = float(supply.loc[sup_mask].sum(numeric_only=True).sum()) / 1e6
        else:
            supply_twh = float(supply.loc[sup_mask].sum()) / 1e6

    total_mio = capex_mio + opex_mio
    return round(total_mio / supply_twh, 2) if supply_twh > 0 else float("nan")

#  Compute hydrogen cost per unit 
uhs_unit = hydrogen_cost_per_unit(uhs)
woUHS_unit = hydrogen_cost_per_unit(woUHS)

#  Bar plot comparison 
labels = ["UHS", "woUHS"]
values = [uhs_unit, woUHS_unit]
colors = [SCENARIO_COLORS[l] for l in labels]

fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(labels, values, color=colors)
ax.set_title("Hydrogen Production Cost per Unit")
ax.set_ylabel("Mio. € / TWh")

for rect, v in zip(bars, values):
    if pd.notna(v):
        ax.text(rect.get_x() + rect.get_width()/2, rect.get_height(),
                f"{v:.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Capacity

In [ ]:
def _sum_h2_electrolysis(series_or_df) -> float:
    """
    Sum entries for carrier == 'H2 Electrolysis' (case-insensitive).
    Works for Series or DataFrame with simple or MultiIndex.
    Returns scalar.
    """
    idx = series_or_df.index
    if isinstance(idx, pd.MultiIndex):
        level = "carrier" if "carrier" in (idx.names or []) else idx.nlevels - 1
        carriers = idx.get_level_values(level).astype(str)
    else:
        carriers = pd.Index(idx).astype(str)

    mask = carriers.str.fullmatch(r"H2 Electrolysis", case=False, na=False)
    sel = series_or_df.loc[mask]
    return float(sel.sum() if isinstance(sel, pd.Series) else sel.sum(numeric_only=True).sum())

def h2_capacity_gw(n: pypsa.Network):
    """
    Returns (installed_GW, expanded_GW, optimal_GW) for H2 Electrolysis on Links.
    """
    inst = n.statistics.installed_capacity(comps="Link")  # MW
    expd = n.statistics.expanded_capacity(comps="Link")   # MW
    opti = n.statistics.optimal_capacity(comps="Link")    # MW

    inst_gw = _sum_h2_electrolysis(inst) / 1e3
    expd_gw = _sum_h2_electrolysis(expd) / 1e3
    opti_gw = _sum_h2_electrolysis(opti) / 1e3
    return inst_gw, expd_gw, opti_gw

#  Compute capacities (GW) 
uhs_inst, uhs_expd, uhs_opti = h2_capacity_gw(uhs)
woUHS_inst, woUHS_expd, woUHS_opti = h2_capacity_gw(woUHS)

#  Prepare grouped bar data 
categories = ["Installed Capacity", "Expanded Capacity", "Optimal Capacity"]
uhs_vals  = [uhs_inst,  uhs_expd,  uhs_opti]
wouhs_vals = [woUHS_inst, woUHS_expd, woUHS_opti]

x = np.arange(len(categories))
width = 0.38

fig, ax = plt.subplots(figsize=(8, 4.5))
b1 = ax.bar(x - width/2, uhs_vals,  width, label="UHS", color=SCENARIO_COLORS["UHS"])
b2 = ax.bar(x + width/2, wouhs_vals, width, label="woUHS", color=SCENARIO_COLORS["woUHS"])

ax.set_title("H₂ Electrolysis Capacities – UHS vs woUHS")
ax.set_ylabel("Capacity (GW)")
ax.set_xticks(x, categories, rotation=15, ha="right")
ax.legend()

# value labels
for bars in (b1, b2):
    for rect in bars:
        v = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, v, f"{v:.2f}",
                ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

##### Hourly Export Prices

In [ ]:
def get_h2_export_prices(n: pypsa.Network):
    """
    Extract hourly marginal prices for the hydrogen export bus.
    Returns a pandas Series with time index.
    """
    h2_buses = n.buses[n.buses.index.str.contains("H2", case=False)].index
    prices = n.buses_t.marginal_price[h2_buses]
    if "H2 export bus" in prices.columns:
        return prices["H2 export bus"]
    else:
        export_cols = [c for c in prices.columns if "export" in c.lower()]
        return prices[export_cols[0]] if export_cols else None

#  Extract hourly prices 
uhs_export_prices = get_h2_export_prices(uhs)
woUHS_export_prices = get_h2_export_prices(woUHS)

#  Plot both on the same axis 
fig, ax = plt.subplots(figsize=(20, 6))

uhs_export_prices.plot(ax=ax, lw=1, color=SCENARIO_COLORS["UHS"], label="UHS")
woUHS_export_prices.plot(ax=ax, lw=1, color=SCENARIO_COLORS["woUHS"], label="woUHS")

ax.set_title("Hourly Hydrogen Export Prices – UHS vs woUHS")
ax.set_xlabel("Time")
ax.set_ylabel("€/MWh")
ax.legend()

plt.tight_layout()
plt.show()

##### Hourly Export Quantity

In [ ]:
def get_h2_export_quantity(n: pypsa.Network):
    """
    Extract aggregated hourly hydrogen export quantity from H2 export links.
    Returns a pandas Series (time-indexed).
    """
    # Filter hydrogen links
    h2_links = n.links[n.links["carrier"].str.contains("H2", case=False, na=False)]
    if h2_links.empty:
        return None

    h2_q = n.links_t.p1[h2_links.index].copy()

    # Identify columns that represent exports (adjust pattern if needed)
    export_cols = [col for col in h2_q.columns if "export" in col.lower()]
    if not export_cols:
        return None

    # Sum all export flows into one series
    h2_q["H2 export"] = h2_q[export_cols].sum(axis=1)
    return h2_q["H2 export"]

#  Extract export quantities 
uhs_export_q = get_h2_export_quantity(uhs)
woUHS_export_q = get_h2_export_quantity(woUHS)

#  Plot side-by-side 
fig, axes = plt.subplots(2, 1, figsize=(20, 8), sharex=True, sharey=True)

if uhs_export_q is not None:
    uhs_export_q.plot(ax=axes[0], lw=1, color=SCENARIO_COLORS["UHS"])
    axes[0].set_title("UHS – Hourly Hydrogen Export Quantity")
    axes[0].set_ylabel("MWh")

if woUHS_export_q is not None:
    woUHS_export_q.plot(ax=axes[1], lw=1, color=SCENARIO_COLORS["woUHS"])
    axes[1].set_title("woUHS – Hourly Hydrogen Export Quantity")
    axes[1].set_xlabel("Time")
    axes[1].set_ylabel("MWh")

plt.tight_layout()
plt.show()

In [ ]:
# aggregate export quantitys to monthly sums and plot them for uhs and woUHS next to each other

In [ ]:
# Plot uhs storage levels over the year

In [ ]:
# plot hydrogen capacity for uhs and woUHS 2050 next to each other

##### Hourly Export Revenue

In [ ]:
def get_h2_export_prices(n: pypsa.Network) -> pd.Series | None:
    """
    Return hourly marginal prices (€/MWh) for the hydrogen export bus.
    """
    h2_buses = n.buses[n.buses.index.str.contains("H2", case=False, na=False)].index
    if len(h2_buses) == 0:
        return None

    prices = n.buses_t.marginal_price[h2_buses]
    if "H2 export bus" in prices.columns:
        return prices["H2 export bus"]
    # fallback: first H2 bus containing 'export'
    export_cols = [c for c in prices.columns if "export" in c.lower()]
    return prices[export_cols[0]] if export_cols else None

def get_h2_export_quantity(n: pypsa.Network) -> pd.Series | None:
    """
    Return aggregated hourly H2 export quantity (MWh) by summing all export links.
    """
    h2_links = n.links[n.links["carrier"].str.contains("H2", case=False, na=False)]
    if h2_links.empty:
        return None

    q = n.links_t.p1[h2_links.index].copy()
    export_cols = [c for c in q.columns if "export" in c.lower()]
    if not export_cols:
        return None

    q["H2 export"] = q[export_cols].sum(axis=1)
    return q["H2 export"]

def hourly_export_revenue(n: pypsa.Network) -> pd.Series | None:
    """
    Compute hourly export revenue (€) = quantity (MWh) * price (€/MWh).
    Aligns indices safely.
    """
    price = get_h2_export_prices(n)
    qty   = get_h2_export_quantity(n)
    if price is None or qty is None:
        return None

    # Align by intersection of timestamps; reindex quantity to price index like your workflow
    idx = price.index.intersection(qty.index)
    if len(idx) == 0:
        return None

    qty_aligned = qty.reindex(idx)
    price_aligned = price.reindex(idx)
    revenue = qty_aligned * price_aligned  # €
    revenue.name = "Hourly Revenue (€)"
    return revenue

# Compute hourly export revenue series
uhs_rev = hourly_export_revenue(uhs)
wouhs_rev = hourly_export_revenue(woUHS)

# Plot side-by-side 
fig, axes = plt.subplots(2, 1, figsize=(20, 8), sharex=True, sharey=True)

if uhs_rev is not None:
    uhs_rev.plot(ax=axes[0], lw=1, color=SCENARIO_COLORS["UHS"])
    axes[0].set_title("UHS – Hourly Hydrogen Export Revenue")
    axes[0].set_ylabel("€")

if wouhs_rev is not None:
    wouhs_rev.plot(ax=axes[1], lw=1, color=SCENARIO_COLORS["woUHS"])
    axes[1].set_title("woUHS – Hourly Hydrogen Export Revenue")
    axes[1].set_xlabel("Time")
    axes[1].set_ylabel("€")

plt.tight_layout()
plt.show()

In [ ]:
# Show how hydrogen underground storage can both stabilize fluctuating hydrogen supply and better handle fluctuating demand (possibly with a graph showing supply/demand coverage with fluctuating supply and demand with/without hydrogen storage)

In [ ]:
# Show how storage can have a price-stabilizing effect on both sides (exporting/importing countries)

In [ ]:
# identify the LCOH of the deposits and investigating which ones have been selected

#### Show PV and Wind Potential

In [ ]:
solar = xr.open_dataset(solar_path)
wind = xr.open_dataset(onwind_path)

def plot_voronoi(n, carrier, voronoi, cmap, projection, title=None, filename=None):
    g = n.generators.loc[n.generators.carrier == carrier]
    br = gpd.read_file(f"../../../pypsa-earth/resources/UHS/bus_regions/regions_{voronoi}.geojson").set_index("name")
    br_area = br.to_crs("ESRI:54009")
    br_area = br_area.geometry.area * 1e-6
    br["p_nom_max"] = g.groupby("bus").sum().p_nom_max / br_area

    fig, ax = plt.subplots(figsize=(8, 4), subplot_kw={"projection": projection})
    plt.rcParams.update({"font.size": 10})
    br.plot(
        ax=ax,
        column="p_nom_max",
        transform=ccrs.PlateCarree(),
        linewidth=0.25,
        edgecolor="k",
        cmap=cmap,
        vmin=0,
        vmax=br["p_nom_max"].max(),
        legend=True,
        legend_kwds={"label": r"potential density"},
    )
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS.with_scale("110m"))
    ax.set_extent(country_coordinates, crs=ccrs.PlateCarree()) 
    
    if title is not None:
        plt.title(title)

#### Wind potentials

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
logging.getLogger("pypsa.io").setLevel(logging.ERROR)
plot_voronoi(
    pypsa.Network(network_path),
    "onwind",
    "onshore",
    "Blues",
    ccrs.PlateCarree(),
    title="Onshore Wind Potential Density [MW/km2]",
)
warnings.simplefilter(action='default', category=FutureWarning)

#### Solar potentials

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
logging.getLogger("pypsa.io").setLevel(logging.ERROR)
plot_voronoi(
    pypsa.Network(network_path),
    "solar",
    "onshore",
    "OrRd",
    ccrs.PlateCarree(),
    title="Solar Photovoltaic Potential Density [MW/km2]",
)
warnings.simplefilter(action='default', category=FutureWarning)

#### Sensitivity Analysis

In [ ]:
# Compare the use of underground hydrogen storage for UHS and woUHS 2050 with various spatial scenarios (1-10-24-100 nodes)

In [ ]:
# compare price stabilizing effect results with and without temporal matching activated